User database created, now a table for the information that is stored

In [3]:
CREATE TABLE users(
    UserID INT PRIMARY KEY IDENTITY(1,1),
    Username NVARCHAR(50) NOT NULL UNIQUE,
    Password NVARCHAR(256) NOT NULL,
    image NVARCHAR(max) NOT NULL
)

Commands completed successfully.

Total execution time: 00:00:00.013

In [2]:
DROP Table users

Commands completed successfully.

Total execution time: 00:00:00.017

A second table is needed to hold the information about the photos being stored. However, as we are storing this as a filestream we must first ensure that we have a filegroup ready.

In [ ]:
-- Create a FILESTREAM filegroup
ALTER DATABASE Users
ADD FILEGROUP PhotosFileGroup
CONTAINS FILESTREAM;

In [13]:
-- Create a FILESTREAM container
ALTER DATABASE Users
ADD FILE (
    NAME = PhotosContainer,
    FILENAME = 'C:\DissertationPhotos'
)
TO FILEGROUP PhotosFileGroup;

Commands completed successfully.

Total execution time: 00:00:00.056

In [15]:
CREATE TABLE Images
(
    ImageID UNIQUEIDENTIFIER ROWGUIDCOL UNIQUE NOT NULL DEFAULT NEWID(),
    FileName NVARCHAR(255),
    ImageData VARBINARY(MAX) FILESTREAM
);

Commands completed successfully.

Total execution time: 00:00:00.017

In [1]:
CREATE TABLE ImagesTwo
(
    ImageTwoID UNIQUEIDENTIFIER ROWGUIDCOL UNIQUE NOT NULL DEFAULT NEWID(),
    FileName NVARCHAR(255),
    ImageTwoData NVARCHAR(max)
)

Commands completed successfully.

Total execution time: 00:00:00.061

As we are currently just b64 encoding our images - there is currently no need for any of the tables and Users needs to be restructured.

In [1]:
DROP TABLE Users
DROP Table ImagesTwo
DROP TABLE Images


Commands completed successfully.

Total execution time: 00:00:00.029

A new table is needed which will contain the UserID and Keystroke pattern of each user each time they sign in.

In [2]:
CREATE TABLE Keystrokes(
    PatternNumber INT PRIMARY KEY NOT NULL IDENTITY(1,1),
    UserID INT NOT NULL,
    Keystrokes NVARCHAR(max),
    FOREIGN KEY (UserID) REFERENCES Users(UserID)

)

Commands completed successfully.

Total execution time: 00:00:00.025

Change the sign up query within the

In [10]:
CREATE PROCEDURE UserSignUp(
    @Username NVARCHAR(50),
    @Password NVARCHAR(512),
    @Image NVARCHAR(max),
    @Keystrokes NVARCHAR(max)
)

AS
BEGIN
    BEGIN TRANSACTION

        BEGIN TRY
            DECLARE @Error NVARCHAR(Max);
            DECLARE @UserID INT;
            BEGIN
                INSERT INTO users (Username, Password, image) VALUES (@Username, @Password, @Image)
                SET @UserID = (SELECT UserID FROM users WHERE (Username = @Username))
                INSERT INTO Keystrokes (UserID, Keystrokes) VALUES (@UserID, @Keystrokes)
            END
            IF @@TRANCOUNT > 0 COMMIT;
        END TRY
        
        BEGIN CATCH
            IF @@TRANCOUNT > 0
                ROLLBACK TRANSACTION
            RAISERROR(@Error,1,0);
        END CATCH;
END;

Commands completed successfully.

Total execution time: 00:00:00.013

Stored Procedure for user signing in

In [15]:
CREATE PROCEDURE UserSignIn(
    @UserID int,
    @Keystrokes NVARCHAR(max)
)

AS
BEGIN
    BEGIN TRANSACTION

        BEGIN TRY
            DECLARE @Error NVARCHAR(Max);
            BEGIN
                INSERT INTO Keystrokes (UserID, Keystrokes) VALUES (@UserID, @Keystrokes)
            END
            IF @@TRANCOUNT > 0 COMMIT;
        END TRY
        
        BEGIN CATCH
            IF @@TRANCOUNT > 0
                ROLLBACK TRANSACTION
            RAISERROR(@Error,1,0);
        END CATCH;
END;

Commands completed successfully.

Total execution time: 00:00:00.004

In [13]:
DROP PROCEDURE UserSignIn

Commands completed successfully.

Total execution time: 00:00:00.011